# Исследование надежности заемщиков.

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Необходимо ответить на вопросы:
* 1 Есть ли зависимость между количеством детей и возвратом кредита в срок?
* 2 Есть ли зависимость между семейным положением и возвратом кредита в срок?
* 3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* 4 Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

df.info()

display(df.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

Перед заполнением пропусков я считаю что нужно привести значение в колонке 'days_employed' к одной размерноасти так как, медианна будет браться с отрицательными числами и она окажеться не верной.

In [3]:
df['days_employed'] = abs(df['days_employed']) #Делаю числа положительными

In [4]:
print(df.isna().sum()) 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Я вижу пропуски в количественных переменных "days_employed" и "total_income" 
Доля пропусков состовляет ~ 10% от общего датасета
Буду заполнять их медианой, так как среднее может не отражать действительности из-за большой разности в числах.
Возможные причины пропусков этих категорий могут быть связаны с тем что пенсионеры могут не указать свой стаж или доход так как фактически уже не работают.


In [5]:
# Заполняью пропуски медианным значением 
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median()) 

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [6]:
display(df.groupby('children')['children'].count()) 

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Я вижу аномальные значение -1 и 20, по моему мнению они не отрожат действительность и я считаю это опечаткой.
Так как я считаю что это опечаткой то думаю что -1 это 1 а 20 это 2, заменю эти значения 

In [7]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2
display(df.groupby('children')['children'].count()) 

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.

Так как по моему предположению цифры до точки в столбце "days_employed" это дни, а после точки идут часы, я просто оставляю цифры до точки.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
df['days_employed'] = df['days_employed'].astype('int') 

In [10]:
df['total_income'] = df['total_income'].astype('int') 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [12]:
df.duplicated().sum() # считаю явные дубликаты

54

In [13]:
df = df.drop_duplicates().reset_index(drop=True) # Удаляю их 

In [14]:
print(df.duplicated().sum())  # Проверяю 

0


In [15]:
df = df.sort_values('education') # Просмотр уникальных значений в столбце 'education'
print(df['education'].unique())

['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']


In [16]:
df['education'] = df.education.str.lower() # приведения их к одному регистру
print(df['education'].unique())

['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']


Я выбрал стандартный метод поиск и удаления дубликатов, с начало я посчитал явные и удалил их, потом проверил важные для моего анализа категоральные столбцы и проверил их на уникальные значения, в столбце 'education' были обнаружены дубликаты, одни и теже категории но написанны разным регистром , появлением таких дубликатов является человеческий фактор, про анализировав что опечатка связанна только с регистром, я привел все значения данного столбца к нижнему регистру тем самым убрав неявные дубликаты.

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаю сводную таблицу 'education' и  'education_id' для того что бы в дата фрейме оставить только id этих значений 

In [17]:
data_education = df.pivot_table(index=['education'], values = 'education_id'  ).sort_values(by='education_id')
display(data_education)

,education_id
education,
высшее,0
среднее,1
неоконченное высшее,2
начальное,3
ученая степень,4


Тоже проделываю для столбцов 'family_status',  'family_status_id'

In [18]:
data_family_status = df.pivot_table(index=['family_status'], values = 'family_status_id' ).sort_values(by='family_status_id')
display(data_family_status)

,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
Не женат / не замужем,4


Удаляю ненужные столбцы из основного датафрейма

In [19]:
df = df.drop(['family_status', 'education' ], axis = 1)
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
20846,0,5301,47,0,4,F,компаньон,0,253236,строительство недвижимости
20755,3,2082,40,0,0,F,компаньон,0,401253,автомобили
19462,1,329699,63,0,0,F,пенсионер,0,180400,операции с недвижимостью
16897,0,3521,48,0,3,M,компаньон,0,212497,на покупку автомобиля
3849,0,5038,59,0,0,M,компаньон,0,165097,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...
21470,2,1984,40,1,0,F,сотрудник,0,82047,на покупку автомобиля
6547,0,5352,58,4,0,M,сотрудник,0,268411,заняться высшим образованием
21465,1,2351,37,4,3,M,сотрудник,0,115949,покупка коммерческой недвижимости
12769,0,376276,62,4,0,F,пенсионер,0,255425,покупка жилой недвижимости


# Шаг 2.6. Категоризация дохода.

Далее необходимо написать функйцию которая назвначит категории разным уровням доходам. условия указываем согласно описания.

In [20]:
def category_income(income):
    
    total_income = income['total_income']
    
    if total_income <= 30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >= 50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'



Вызываем фукцию и результат ее выполнения добавляем в новую колонку 'total_income_category'.

In [21]:
df['total_income_category'] =  df.apply(category_income, axis=1) 
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
20846,0,5301,47,0,4,F,компаньон,0,253236,строительство недвижимости,B
20755,3,2082,40,0,0,F,компаньон,0,401253,автомобили,B
19462,1,329699,63,0,0,F,пенсионер,0,180400,операции с недвижимостью,C
16897,0,3521,48,0,3,M,компаньон,0,212497,на покупку автомобиля,B
3849,0,5038,59,0,0,M,компаньон,0,165097,покупка недвижимости,C
...,...,...,...,...,...,...,...,...,...,...,...
21470,2,1984,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C
6547,0,5352,58,4,0,M,сотрудник,0,268411,заняться высшим образованием,B
21465,1,2351,37,4,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,C
12769,0,376276,62,4,0,F,пенсионер,0,255425,покупка жилой недвижимости,B


### Шаг 2.7. Категоризация целей кредита.

Перед тем как написать функцию которая будет формировать категории из целей кредита, выведем все уникальные значение столбца 'purpose' для того что бы видеть какие уникальные цели указаны в датасете.

In [22]:

purpose_group_category = df['purpose'].value_counts()
print(purpose_group_category)

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Создаем функцию которая более конкретно категоризирует цели. 

In [3]:
def category(row):
    category = row['purpose']
    if  'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'
    

Выглядит конечно не читабельно, но я не смог придумать как еще указать условия более корректно.

Результат функции добавляем в таблицу в новую колонку 'purpose_category'

In [24]:
df['purpose_category'] = df.apply(category, axis=1 )
display(df.head(20))


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
20846,0,5301,47,0,4,F,компаньон,0,253236,строительство недвижимости,B,операции с недвижимостью
20755,3,2082,40,0,0,F,компаньон,0,401253,автомобили,B,операции с автомобилем
19462,1,329699,63,0,0,F,пенсионер,0,180400,операции с недвижимостью,C,операции с недвижимостью
16897,0,3521,48,0,3,M,компаньон,0,212497,на покупку автомобиля,B,операции с автомобилем
3849,0,5038,59,0,0,M,компаньон,0,165097,покупка недвижимости,C,операции с недвижимостью
4799,0,2111,29,0,4,F,сотрудник,0,270468,профильное образование,B,получение образования
2775,0,1453,32,0,0,F,сотрудник,0,87599,жилье,C,операции с недвижимостью
8555,0,1136,28,0,1,M,компаньон,0,466041,на проведение свадьбы,B,проведение свадьбы
11314,1,575,41,0,3,F,сотрудник,0,127656,дополнительное образование,C,получение образования
6864,0,1773,33,0,4,M,сотрудник,0,241419,покупка жилой недвижимости,B,операции с недвижимостью


### Ответы на вопросы.

Что бы наглядно продемонстрировать зависимости, составим таблицу согласно поставленому вопросу, зависимость я буду вырожать в процентах проссрочки от общего числа долгов по каждой категории. 

Описание таблицы
'0' - часть людей у которых нет просрочки.
'1' - часть людей у которых есть просрочка.
'all_debts' - общее число кредитов.
'procent'- зависимость выраженная в процентах от общего числа кридитов.

# 1 Отвечаем на впорос: есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
debt_children = df.pivot_table(index = ['children'], columns = 'debt', values= 'total_income_category',  aggfunc='count' )
debt_children['all_debts'] = debt_children[1]+debt_children[0]
debt_children['procent'] = round(debt_children[1] / debt_children['all_debts']*100,2)
display(debt_children)

debt,0,1,all_debts,procent
children,,,,
0,13044.0,1063.0,14107.0,7.54
1,4411.0,445.0,4856.0,9.16
2,1926.0,202.0,2128.0,9.49
3,303.0,27.0,330.0,8.18
4,37.0,4.0,41.0,9.76
5,9.0,NaN,NaN,NaN


Как можно увидить по данной таблице люди не имеющие детей гораздо реже просрачивают выплату, немного лучше себя показывает группа людей у которых 3 ребенка, а у много детных людей просрочеки и вовсе отсутсвуют, самая рисковая группа людей являются те у которых 2 ребенка в семье.

# 2 Отвечаем на впорос: есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
debt_children = df.pivot_table(index = ['family_status_id'], columns = 'debt', values= 'total_income_category',  aggfunc='count' )
debt_children['all_debts'] = debt_children[1]+debt_children[0]
debt_children['procent'] = debt_children[1] / debt_children['all_debts']*100
display(debt_children)

debt,0,1,all_debts,procent
family_status_id,,,,
0,11413,931,12344,7.542126
1,3775,388,4163,9.320202
2,896,63,959,6.569343
3,1110,85,1195,7.112971
4,2536,274,2810,9.750890


In [27]:
data_family_status # Выведем отдельно таблицу с рассшифровкой значений семейного статуса.

,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
Не женат / не замужем,4


Тут мы можем наблюдать что самыми надежными клиентами банка являются люди имеющие семейный статус "вдовец / вдова", далее неплохую статистику показывают 2 группы людей "женат / замужем" и "в разводе", самымы рисковыми клиентами являются люди находящиеся в группе "Не женат / не замужем".

# 3 Отвечаем на впорос: есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
debt_children = df.pivot_table(index = ['total_income_category'], columns = 'debt', values= 'income_type',  aggfunc='count' )
debt_children['all_debts'] = debt_children[1]+debt_children[0]
debt_children['procent'] = round(debt_children[1] / debt_children['all_debts']*100,2)
display(debt_children)

debt,0,1,all_debts,procent
total_income_category,,,,
A,23,2,25,8.00
B,4685,356,5041,7.06
C,14673,1360,16033,8.48
D,329,21,350,6.00
E,20,2,22,9.09


Расшифровка категорий дохода.
1000001 и выше — 'A'
200001–1000000 — 'B'
50001–200000 — 'C'
30001–50000 — 'D'
0–30000 — 'E'

По данной таблице можно увидеть что люди с категорией дохода "D" являются самыми надежными клиентами, а люди с самой низжей категорией являются самыми рискованными и то что люди имеющие самый большой доход по категориям, не сильно отличаются от людей со среднем доходом по категориям. 

# 4 Отвечаем на впорос: как разные цели кредита влияют на его возврат в срок?

In [29]:
debt_children = df.pivot_table(index = ['purpose_category'], columns = 'debt', values= 'income_type',  aggfunc='count' )
debt_children['all_debts'] = debt_children[1]+debt_children[0]
debt_children['procent'] = round(debt_children[1] / debt_children['all_debts']*100,2)
display(debt_children)

debt,0,1,all_debts,procent
purpose_category,,,,
операции с автомобилем,3905,403,4308,9.35
операции с недвижимостью,10032,782,10814,7.23
получение образования,3644,370,4014,9.22
проведение свадьбы,2149,186,2335,7.97


Тут на мой взгляд таблица поделилась пополам так как отличие состаляют десятые процента, люди берущие кредит на жилье и свадьбу самые ответсвенны, а получение образование и операции с авто самые рисковые категории кридитования

ИМХО: считаю последнюю таблицу самой не объективной, так как нет конкретных сумм и нельзя объективно судить, к примеру покупка жилья может стоить в 10ки раз больше каких либо операций с автомобилем или получения образования, но ущерб банку сложно оценить, так как вероятнее при просрочке оплаты долга за жилье, банк скорее всего станет собственником этого жилья что может принести ему доход нежеле чем убытки (изхожу из своего дилетантский мнения). В то же время свадьба вероятнее будет стоить больше получения образования. По этому я бы исключил эту таблицу из анализа.

##### Вывод 1:

## Общий вывод:

В задании был проведен анализ групп заемщиков с разынми целями кредитования и разными условиями существования, в ответах на вопросы наглядно показанно при каких зависимостях какие группы заемщиков являются более благонадежными, а какие на оборот. Изходя из низ банк может делать выводы о своем клиенте стоит ему выдавать кредит или нет или же предоставлять своим клиентам разного плана специальные условия кредитования.

Из ходя из анализа базы данных можно сделать представление о самом не надежном клиенте банка.
* Cемейное положение : Не женат / не замужем или гражданский брак 
* Наличие детей: 2 или 4 
* Уровень дохода: от 0 до 30к р
* Цель кредитования: опереции с автомобилем или же на образование.